In [1]:
# ML MODEL RANDOM FOREST- feature based prediction
# load dataset
import pandas as pd
df=pd.read_excel("mutation_data.csv.xlsx")
print(df.head())

   wild_seq   mut_seq        Label
0  ATGCGTAA  ATGAGTAA      Neutral
1  GCTTACGA  GCGTACGA  Deleterious
2  TTGCCGAA  TTGCCAAA   Beneficial
3  ATGGGCAA  ATGAGCAA      Neutral
4  TTTCCGAA  TTTGCGAA  Deleterious


In [2]:
def gc_content(seq):
  return (seq.count('G') + seq.count('C')) / len(seq)

In [3]:
features = []
for i, row in df.iterrows():
    wild = row['wild_seq']
    mut = row['mut_seq']
    pos = next(i for i in range(len(wild)) if wild[i] != mut[i])
    gc_diff = gc_content(mut) - gc_content(wild)
    features.append([pos, gc_diff])

In [4]:
import numpy as np
X = pd.DataFrame(features, columns=['mutation_pos', 'gc_diff'])
y = df['Label']
print(X.head())

   mutation_pos  gc_diff
0             3   -0.125
1             2    0.125
2             5   -0.125
3             3   -0.125
4             3    0.000


In [5]:
#model training
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [8]:
#test model
y_pred = rf_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.5

Classification Report:
               precision    recall  f1-score   support

 Deleterious       0.00      0.00      0.00         1
     Neutral       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



C:\Users\gangw.LAPTOP-92MA42BC\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\gangw.LAPTOP-92MA42BC\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\gangw.LAPTOP-92MA42BC\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` par

In [9]:
#prediction test
test = [[3, -0.125]]
print("Predicted Label:", rf_model.predict(test))

Predicted Label: ['Neutral']


C:\Users\gangw.LAPTOP-92MA42BC\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [10]:
# deep learning model(sequence based prediction)
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [11]:
#dna sequencing encoding
def encode_seq(seq):
    mapping = {'A':0, 'T':1, 'G':2, 'C':3}
    return [mapping[base] for base in seq]

In [12]:
#preparing input data
X_seq = []
for i, row in df.iterrows():
    wild = encode_seq(row['wild_seq'])
    mut = encode_seq(row['mut_seq'])
    combined = wild + mut  # combine both sequences
    X_seq.append(combined)
X_seq = np.array(X_seq)

In [13]:
#label encoding
le = LabelEncoder()
y_encoded = le.fit_transform(df['Label'])
y_encoded = to_categorical(y_encoded)

In [14]:
#sequence padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_seq = pad_sequences(X_seq, maxlen=16, padding='post')

In [15]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_encoded, test_size=0.2, random_state=42)

In [16]:
#build LSTM model
model = Sequential([
    Embedding(input_dim=4, output_dim=8, input_length=16),
    LSTM(32, return_sequences=False),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

C:\Users\gangw.LAPTOP-92MA42BC\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
#train model
history = model.fit(X_train, y_train, epochs=30, batch_size=4, validation_data=(X_test, y_test))

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 911ms/step - accuracy: 0.2500 - loss: 1.0978 - val_accuracy: 0.0000e+00 - val_loss: 1.1020
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.3750 - loss: 1.0967 - val_accuracy: 0.0000e+00 - val_loss: 1.1027
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.3750 - loss: 1.0961 - val_accuracy: 0.0000e+00 - val_loss: 1.1037
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.3750 - loss: 1.0954 - val_accuracy: 0.0000e+00 - val_loss: 1.1060
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.3750 - loss: 1.0945 - val_accuracy: 0.0000e+00 - val_loss: 1.1065
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.3750 - loss: 1.0937 - val_accuracy: 0.0000e+00 - val_loss: 1.1072
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.3750 - loss: 1.0934 - val_accuracy: 0.0000e+00 - val_loss: 1.1095
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.3750 - loss: 1.0925 - val_

In [18]:
#evaluate model
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.0000e+00 - loss: 1.1720
Test Accuracy: 0.00


In [19]:
#prediction on new mutation
wild = "ATGCGTAA"
mut = "ATGAGTAA"
combined = encode_seq(wild) + encode_seq(mut)
combined = pad_sequences([combined], maxlen=16, padding='post')
pred = model.predict(combined)
pred_label = le.inverse_transform([np.argmax(pred)])
print("Predicted Label:", pred_label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 451ms/step
Predicted Label: Beneficial


In [20]:
#LLM DistilGPT2- explanation generator
from transformers import pipeline

In [21]:
from transformers import pipeline
generator = pipeline("text-generation", model="distilgpt2", framework="pt")
result = generator("Explain protein folding in simple terms:", max_length=80)
print(result[0]["generated_text"])

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explain protein folding in simple terms:



"We call this the same protein folding system that we use in our biology," said Dr. Michael Carleton, assistant professor of medicine and biology. "We call it the same protein folding system that we use in our biology, and we call it the same protein folding system that we use in our biology, and we call it the same protein folding system that we use in our biology, and we call it the same protein folding system that we use in our biology, and we call it the same protein folding system that we use in our biology, and we call it the same protein folding system that we use in our biology, and we call it the same protein folding system that we use in our biology, and we call it the same protein folding system that we use in our biology, and we call it the same protein folding system that we use in our biology, and we call it the same protein folding system that we use in our biology, and we call it the same protein folding system that we use in 

In [22]:
#define function for explanation
def explain_mutation(wild, mut, label):
    prompt = f"Explain why the DNA mutation from {wild} to {mut} might be {label}. Biological reason: "
    result = generator(prompt, max_length=50, temperature=0.7, num_return_sequences=1)
    explanation = result[0]['generated_text']
    return explanation

In [23]:
#test on model output
wild = "ATGCGTAA"
mut = "ATGAGTAA"
label = "Beneficial"
print(explain_mutation(wild, mut, label))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Explain why the DNA mutation from ATGCGTAA to ATGAGTAA might be Beneficial. Biological reason:   A previous paper, based on an unpublished genome study on ATGCGTAA and ATGAGTAA, suggests that there is a direct pathogen of ATGAGTAA in the human body that is highly affected by the disease, and that the mutation in at least one of the genes is also beneficial. It is important to note that ATGCGTAA is also responsible for the growth of the immune system in the human body. In contrast, the ability of the immune system to fight the disease and to control the disease is unknown. It is important to note that ATGCGTAA was found in the body, and that the immune system is affected by the disease. This is a particularly important question.


The key question is: What is the difference between ATGCGTAA and ATGAGTAA in the human body? That is, the immune-system is not affected by ATGCGTAA.
The cellular cell is more complex than the cell itself : it is the immune system. It is the cell that is respon

In [24]:
#combine with model output auto
wild = "ATGCGTAA"
mut = "ATGAGTAA"
combined = encode_seq(wild) + encode_seq(mut)
combined = pad_sequences([combined], maxlen=16, padding='post')
pred = model.predict(combined)
pred_label = le.inverse_transform([np.argmax(pred)])[0]
exp = explain_mutation(wild, mut, pred_label)
print(f"🧬 Predicted Label: {pred_label}")
print(f"🧠 Explanation: {exp}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🧬 Predicted Label: Beneficial
🧠 Explanation: Explain why the DNA mutation from ATGCGTAA to ATGAGTAA might be Beneficial. Biological reason: ???.


In [25]:
model.save("lstm_model.h5")
print("Saved lstm_model.h5")

Saved lstm_model.h5


In [26]:
import joblib
joblib.dump(rf_model, "ml_model.joblib")
joblib.dump(le, "label_encoder.joblib")

['label_encoder.joblib']

In [27]:
model.save("lstm_model.h5")
import joblib
joblib.dump(le, "label_encoder.joblib")

['label_encoder.joblib']

In [28]:
from transformers import BertTokenizer, BertForSequenceClassification
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model.save_pretrained("llm_model")
tokenizer.save_pretrained("llm_model")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('llm_model\\tokenizer_config.json',
 'llm_model\\special_tokens_map.json',
 'llm_model\\vocab.txt',
 'llm_model\\added_tokens.json')

In [29]:
from transformers import BertForSequenceClassification, BertTokenizer
model_path = "./llm_model"
bert_model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

In [30]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("./llm_model")
bert_model = BertForSequenceClassification.from_pretrained("./llm_model")

In [31]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
# Load model and tokenizer
tokenizer = BertTokenizer.from_pretrained("./llm_model")
model = BertForSequenceClassification.from_pretrained("./llm_model")
# Example text
text = "I love working with machine learning models!"
# Tokenize
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
# Run model
with torch.no_grad():
    outputs = model(**inputs)
# Get prediction
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()
print("Predicted class:", predicted_class)

Predicted class: 0
